In [1]:
import os

In [2]:
%pwd

'c:\\Users\\satish.hiremath\\OneDrive - o9 Solutions\\Desktop\\python\\e2e_mlops_dimond_price_prediction\\e2e_mlops_dimond_price_prediction\\experiments'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    data_dir:Path
    updated_base_model_path: Path
    model_param_file:Path
    target_column:str
    drop_column:list
    test_train_split:int
    random_state:int

    

In [5]:
from src.DiamondPricePredictor.constants import *
from src.DiamondPricePredictor.utils.common import read_yaml, create_directories,load_json,save_loaded_json,save_json,save_obj

[ 2024-02-02 22:43:30,617] 24 root-INFO -Logging has started


In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        model_params_filepath = MODEL_PARAM_FILE_PATH,
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.model_param=load_json(model_params_filepath)
        

        
        create_directories([self.config.artifacts_root,self.config.artifacts_root+'/prepare_base_model'])

        path=self.config.artifacts_root+'/prepare_base_model'+'/model_params.json'
    
        save_loaded_json(path,self.model_param)


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            data_dir=Path(config.data_dir),
            model_param_file=Path(config.model_param_file),
            target_column=self.params.TARGET_COLUMN,
            drop_column=self.params.DROP_COLUMN,
            test_train_split=self.params.TEST_TRAIN_SPLIT,
            random_state=self.params.RANDOM_STATE,
        )

        return prepare_base_model_config


In [7]:
import os
import urllib.request as request
from zipfile import ZipFile
from sklearn.metrics import r2_score

In [8]:
from src.DiamondPricePredictor.utils.MLUtils import PrepareData,SplitDataSet,evaluate_model,DataTransformation,best_score
from src.DiamondPricePredictor.exception import CustomException
import json
import sys
from sklearn.tree import DecisionTreeRegressor

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def initiate_model_trainer(self):
        try:            
            path_of_files=self.config.data_dir
            Target_Column=self.config.target_column
            Drop_Columns=self.config.drop_column
            split_ratio=self.config.test_train_split
            random_state=self.config.random_state
            model_param_file=self.config.model_param_file
            base_model_path=self.config.base_model_path
            root_dir=self.config.root_dir


            feature_data,Target_data=PrepareData(Path(path_of_files),Target_Column,Drop_Columns)
            
            
            x_train,y_train,x_test,y_test=SplitDataSet(feature_data,Target_data,split_ratio,random_state)
            
    
            
            x_train_t,y_train_t,x_test_t,y_test_t=DataTransformation(x_train,y_train,x_test,y_test)

            model_and_parameters=load_json(model_param_file)
            
            json_string = json.dumps(model_and_parameters)
            data_dict = json.loads(json_string)
            params = data_dict.get("params", {})
            models=data_dict.get("models", {})



            
            model_report:dict=evaluate_model(x_train=x_train_t,y_train=y_train_t,x_test=x_test_t,y_test=y_test_t,models=models,param=params)

            
            best_model_score=max(sorted(model_report.values()))

            best_model_name=list(model_report.keys())[list(model_report.values()).index(best_model_score)]

            best_model=models[best_model_name]

            if best_model_score<0.6:
                raise CustomException('no best model found')
            

            print(root_dir)
            #save_obj(
            #    file_path=base_model_path,
            #    obj=best_model
            #)
            
            r2_score_best_model=best_score(root_dir,best_model,x_train_t,y_train_t)

            print(r2_score_best_model)

            return r2_score_best_model
        except Exception as e:
           raise CustomException(e,sys)
    


In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.initiate_model_trainer()
except Exception as e:
    raise CustomException(e,sys)

[ 2024-02-02 22:43:40,838] 35 root-INFO -yaml file: config\config.yaml loaded successfully
[ 2024-02-02 22:43:40,855] 35 root-INFO -yaml file: params.yaml loaded successfully
[ 2024-02-02 22:43:40,858] 102 root-INFO -json file loaded succesfully from: src\model_param.json
[ 2024-02-02 22:43:40,866] 55 root-INFO -created directory at: artifacts
[ 2024-02-02 22:43:40,870] 55 root-INFO -created directory at: artifacts/prepare_base_model
artifacts\prepare_base_model\model_params.json
[ 2024-02-02 22:43:40,876] 86 root-INFO -json file saved at: artifacts\prepare_base_model\model_params.json
[ 2024-02-02 22:43:40,881] 55 root-INFO -created directory at: artifacts/prepare_base_model
artifacts\data_ingestion\train.csv
[ 2024-02-02 22:43:41,237] 70 root-INFO -The shape of X_train (154858, 9)
[ 2024-02-02 22:43:41,255] 71 root-INFO -The head of X_test <bound method NDFrame.head of         carat        cut color clarity  depth  table     x     y     z
83475    0.32    Premium     E     SI1   61.6

'c:\\Users\\satish.hiremath\\OneDrive - o9 Solutions\\Desktop\\python\\e2e_mlops_dimond_price_prediction\\e2e_mlops_dimond_price_prediction'